In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import os

import numpy as np
np.random.seed(2016)
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import list_pictures, array_to_img

from image_ext import list_pictures_in_multidir, load_imgs_asarray   

Using TensorFlow backend.


In [2]:
def create_fcn(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv5)

    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv5])
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv7), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(input=inputs, output=conv12)

    return fcn

In [3]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2.*intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)

In [4]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [5]:
if __name__ == '__main__':

    fpaths_xs_train = list_pictures_in_multidir(['./dataset/train/image/', 
                                                 './dataset/train/image-aug/'])
    fpaths_ys_train = list_pictures_in_multidir(['./dataset/train/gt/', 
                                                 './dataset/train/gt-aug/'])
    fpaths_xs_valid = list_pictures_in_multidir(['./dataset/valid/image/', 
                                                 './dataset/valid/image-aug/'])
    fpaths_ys_valid = list_pictures_in_multidir(['./dataset/valid/gt/', 
                                                 './dataset/valid/gt-aug/'])
#    fpaths_xs_train = list_pictures_in_multidir(['./data-scene/train/', 
#                                                 './data-scene/train-aug/'])
#    fpaths_ys_train = list_pictures_in_multidir(['./data-scene/train_mask/', 
#                                                 './data-scene/train_mask-aug/'])
#    fpaths_xs_valid = list_pictures_in_multidir(['./data-scene/valid/', 
#                                                 './data-scene/valid-aug'])
#    fpaths_ys_valid = list_pictures_in_multidir(['./data-scene/valid_mask', 
#                                                 './data-scene/valid_mask-aug/'])
    
    #dim_ordering ='tf' 
    dim_ordering = 'channels_first'
    target_size = (224, 224)
    
    # データを配列として取得
    print('loading data...')
    X_train = load_imgs_asarray(fpaths_xs_train, grayscale=False, 
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_train = load_imgs_asarray(fpaths_ys_train, grayscale=True, 
                                target_size=target_size, dim_ordering=dim_ordering)
    X_valid = load_imgs_asarray(fpaths_xs_valid, grayscale=False,
                                target_size=target_size, dim_ordering=dim_ordering)
    Y_valid = load_imgs_asarray(fpaths_ys_valid, grayscale=True,
                                target_size=target_size, dim_ordering=dim_ordering)
    print('==> ' + str(len(X_train)) + ' training images loaded')
    print('==> ' + str(len(Y_train)) + ' training masks loaded')
    print('==> ' + str(len(X_valid)) + ' validation images loaded')
    print('==> ' + str(len(Y_valid)) + ' validation masks loaded')    

loading data...
........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
    # 前処理
    print('computing mean and standard deviation...')
    mean = np.mean(X_train, axis=(0, 2, 3))
    std = np.std(X_train, axis=(0, 2, 3))
    print('==> mean: ' + str(mean))
    print('==> std : ' + str(std))
    
    fname_stats = 'stats.npy'

    print('saving mean and standard deviation to ' + fname_stats + '...')
    stats = {'mean': mean, 'std': std}
    np.savez(fname_stats, **stats)
    print('==> done')

    print('globally normalizing data...')
    for i in range(3):
        X_train[:, i] = (X_train[:, i] - mean[i]) / std[i]
        X_valid[:, i] = (X_valid[:, i] - mean[i]) / std[i]
    Y_train /= 255
    Y_valid /= 255
    print('==> done')
    
    print(mean.shape)
    print(std.shape)

computing mean and standard deviation...
==> mean: [ 211.48577881  174.08840942  191.61517334]
==> std : [ 59.50221252  58.8382988   59.77029419]
saving mean and standard deviation to stats.npy...
==> done
globally normalizing data...
==> done
(3,)
(3,)


In [7]:
    # モデルを作成
    print('creating model...')
    model = create_fcn(target_size)
    model.summary()

creating model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 224, 224)  896         input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 32, 224, 224)  9248        conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 32, 112, 112)  0           conv2d_2[0][0]                   
_________________________________________________________________________

/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.5/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=Tensor("in...)`


In [8]:
# 損失関数，最適化手法を定義
adam = Adam(lr=1e-5)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])

# 構造・重みを保存するディレクトリーの有無を確認
dpath_checkpoints = './checkpoints/'
if not os.path.isdir(dpath_checkpoints):
    os.mkdir(dpath_checkpoints)

In [9]:
# モデルの構造を保存
json_string = model.to_json()
fname_architecture = 'fcn.json'
fpath_architecture = os.path.join(dpath_checkpoints, fname_architecture)
with open(fpath_architecture, 'w', encoding='utf8') as f:
    f.write(json_string)

In [10]:
print(X_train.shape)
print(Y_train.shape)

(2071, 3, 224, 224)
(2071, 1, 224, 224)


In [12]:
# 重みを保存するためのオブジェクトを用意
fname_weights = 'model_weights_{epoch:02d}.h5'
fpath_weights = os.path.join(dpath_checkpoints, fname_weights)
checkpointer = ModelCheckpoint(filepath=fpath_weights, save_best_only=False)

# トレーニングを開始
print('start training...')
model.fit(X_train, Y_train, batch_size=5, epochs=300, verbose=1, 
          shuffle=True, validation_data=(X_valid, Y_valid), callbacks=[checkpointer])

start training...
Train on 2071 samples, validate on 212 samples
Epoch 1/300
2071/2071 [==============================] - 180s - loss: -0.4684 - dice_coef: 0.4684 - val_loss: -0.7406 - val_dice_coef: 0.7406
Epoch 2/300
2071/2071 [==============================] - 176s - loss: -0.7207 - dice_coef: 0.7207 - val_loss: -0.7485 - val_dice_coef: 0.7485
Epoch 3/300
2071/2071 [==============================] - 177s - loss: -0.7655 - dice_coef: 0.7655 - val_loss: -0.7152 - val_dice_coef: 0.7152
Epoch 4/300
2071/2071 [==============================] - 176s - loss: -0.7926 - dice_coef: 0.7926 - val_loss: -0.6733 - val_dice_coef: 0.6733
Epoch 5/300
2071/2071 [==============================] - 177s - loss: -0.7940 - dice_coef: 0.7940 - val_loss: -0.7745 - val_dice_coef: 0.7745
Epoch 6/300
2071/2071 [==============================] - 177s - loss: -0.8215 - dice_coef: 0.8215 - val_loss: -0.7796 - val_dice_coef: 0.7796
Epoch 7/300
2071/2071 [==============================] - 177s - loss: -0.8350 - dic

2071/2071 [==============================] - 176s - loss: -0.9582 - dice_coef: 0.9582 - val_loss: -0.8056 - val_dice_coef: 0.8056
Epoch 58/300
2071/2071 [==============================] - 176s - loss: -0.9585 - dice_coef: 0.9585 - val_loss: -0.8204 - val_dice_coef: 0.8204
Epoch 59/300
2071/2071 [==============================] - 176s - loss: -0.9587 - dice_coef: 0.9587 - val_loss: -0.8285 - val_dice_coef: 0.8285
Epoch 60/300
2071/2071 [==============================] - 176s - loss: -0.9608 - dice_coef: 0.9608 - val_loss: -0.8036 - val_dice_coef: 0.8036
Epoch 61/300
2071/2071 [==============================] - 176s - loss: -0.9595 - dice_coef: 0.9595 - val_loss: -0.7869 - val_dice_coef: 0.7869
Epoch 62/300
2071/2071 [==============================] - 177s - loss: -0.9601 - dice_coef: 0.9601 - val_loss: -0.8161 - val_dice_coef: 0.8161
Epoch 63/300
2071/2071 [==============================] - 177s - loss: -0.9598 - dice_coef: 0.9598 - val_loss: -0.7813 - val_dice_coef: 0.7813
Epoch 64/300

2071/2071 [==============================] - 176s - loss: -0.9659 - dice_coef: 0.9659 - val_loss: -0.8408 - val_dice_coef: 0.8408
Epoch 115/300
2071/2071 [==============================] - 175s - loss: -0.9672 - dice_coef: 0.9672 - val_loss: -0.8245 - val_dice_coef: 0.8245
Epoch 116/300
2071/2071 [==============================] - 176s - loss: -0.9661 - dice_coef: 0.9661 - val_loss: -0.8379 - val_dice_coef: 0.8379
Epoch 117/300
2071/2071 [==============================] - 176s - loss: -0.9667 - dice_coef: 0.9667 - val_loss: -0.8370 - val_dice_coef: 0.8370
Epoch 118/300
2071/2071 [==============================] - 176s - loss: -0.9673 - dice_coef: 0.9673 - val_loss: -0.8351 - val_dice_coef: 0.8351
Epoch 119/300
2071/2071 [==============================] - 176s - loss: -0.9667 - dice_coef: 0.9667 - val_loss: -0.8278 - val_dice_coef: 0.8278
Epoch 120/300
2071/2071 [==============================] - 176s - loss: -0.9671 - dice_coef: 0.9671 - val_loss: -0.8444 - val_dice_coef: 0.8444
Epoch 

KeyboardInterrupt: 